In [ ]:
import pandas as pd
from matplotlib import pyplot
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

dataset_URL = "https://raw.githubusercontent.com/CheMBurN695/FinCrime_ML_Model/refs/heads/master/Trnx_Dataset.csv?token=GHSAT0AAAAAACYPUMUJ3BSBWUE7ZSIQEECQZ3IO4GQ"
testing_dataset_URL = "https://raw.githubusercontent.com/CheMBurN695/FinCrime_ML_Model/refs/heads/master/Testing_Dataset.csv?token=GHSAT0AAAAAACYPUMUITFWKKFIOTAI22AW4Z3INTRA"
training_dataset_URL = "https://raw.githubusercontent.com/CheMBurN695/FinCrime_ML_Model/refs/heads/master/Training_Dataset.csv?token=GHSAT0AAAAAACYPUMUJBIMGLEC4BVP3JOLMZ3IOOYA"
dataFrame = pd.read_csv(dataset_URL)
encodedDataFrame = pd.read_csv(dataset_URL)
testing_dataFrame = pd.read_csv(testing_dataset_URL)

In [ ]:
sowByCountry = dataFrame.groupby(["Country", "Source of Money"]).size()


In [ ]:
country_count = dataFrame.groupby(["Country"]).size()
illegal_trnxs = dataFrame['Source of Money'] == "Illegal"
illegal_trnx_byCountry = dataFrame[illegal_trnxs].groupby(["Country"]).size()
not_illegal_trnx_byCountry = country_count - illegal_trnx_byCountry
percent_illegal = (illegal_trnx_byCountry / country_count) * 100
percent_legal = (not_illegal_trnx_byCountry / country_count) * 100

_fig, _axes = pyplot.subplots(figsize=(8,6))
_axes.bar(illegal_trnx_byCountry.index, illegal_trnx_byCountry, color='red', label="Illegal")
_axes.bar(not_illegal_trnx_byCountry.index, not_illegal_trnx_byCountry, color='green', label="Legal")

_axes.set_title("Distribution of Illegal Transactions per Country In Dataset")
_axes.set_xlabel("Country")
_axes.set_ylabel("Transaction Count")
_axes.set_xticks(country_count.index)
_axes.set_xticklabels(country_count.index, rotation=45)
_axes.legend()
pyplot.show()


In [ ]:
print(dataFrame.isnull().sum())

In [ ]:

label_encoder = LabelEncoder()
encodedDataFrame['Country'] = label_encoder.fit_transform(dataFrame['Country'])
encodedDataFrame['Destination Country'] = label_encoder.fit_transform(dataFrame['Destination Country'])
encodedDataFrame['Transaction Type'] = label_encoder.fit_transform(dataFrame['Transaction Type'])
encodedDataFrame['Amount (USD)'] = label_encoder.fit_transform(dataFrame['Amount (USD)'])
encodedDataFrame['Person Involved'] = label_encoder.fit_transform(dataFrame['Person Involved'])
encodedDataFrame['Industry'] = label_encoder.fit_transform(dataFrame['Industry'])
encodedDataFrame['Source of Money'] = label_encoder.fit_transform(dataFrame['Source of Money'])

encodedDataFrame['Money Laundering Risk Score'] = label_encoder.fit_transform(dataFrame['Money Laundering Risk Score'])

X = encodedDataFrame[['Industry', 'Country', 'Destination Country']]
Y = encodedDataFrame['Money Laundering Risk Score']


In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X,Y)